<a href="https://colab.research.google.com/github/SamuelPassamani/XCam/blob/main/xcam-colab/XCam%20GIFs/Conversor%20M3U8-GIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

# Monta o Google Drive no ambiente do Colab
drive.mount('/content/drive')

print("✅ Google Drive montado com sucesso!")

Mounted at /content/drive
✅ Google Drive montado com sucesso!


In [ ]:
import os
import requests
import time
import math

# ==============================================================================
# === CONFIGURAÇÕES GLOBAIS ===
# ==============================================================================

# --- Configurações da API ---
API_KEY = "99090882"
PRIMARY_API_URL = f"https://api.xcam.gay/?limit=1500&key={API_KEY}"
SECONDARY_API_URL_TEMPLATE = f"https://api.xcam.gay/?user={{username}}&key={API_KEY}"

# --- Configurações de Saída (Google Drive) ---
DRIVE_FOLDER_PATH = '/content/drive/MyDrive/Projetos/XCam/Conteúdo Social/XCam Social Mídias/XCam GIFs'
LOG_FILE_NAME = "_usuarios_processados.log"

# --- Configurações de Download e Conversão (AJUSTADO PARA CONTROLE DE TAMANHO) ---
# Alvo de tamanho máximo para o GIF final em Megabytes (MB).
TARGET_GIF_SIZE_MB = 15.0

# Fator de segurança para garantir que o GIF final fique ABAIXO do alvo.
# 0.95 = 95% do alvo. Aumente se os GIFs ainda estiverem passando do limite.
SAFETY_FACTOR = 0.95

# Duração máxima do clipe de vídeo a ser baixado para análise.
# O script usará este clipe para calcular a duração ideal do GIF. 30 segundos é um bom valor.
MAX_CLIP_SECONDS_TO_ANALYZE = 30

# Duração mínima para um GIF ser considerado útil (em segundos).
MIN_GIF_DURATION = 2.0

# Qualidade do GIF (mantida dos scripts anteriores)
GIF_FPS = 15
GIF_WIDTH = 640

# ==============================================================================
# === SCRIPT DE AUTOMAÇÃO (Não altere o código abaixo) ===
# ==============================================================================

print("Iniciando o processo de automação com controle de tamanho de arquivo...")

# --- Etapa 1: Preparar Ambiente e Ler Registro ---
os.makedirs(DRIVE_FOLDER_PATH, exist_ok=True)
caminho_log = os.path.join(DRIVE_FOLDER_PATH, LOG_FILE_NAME)
processed_users = set()
try:
    with open(caminho_log, 'r') as f:
        processed_users = {line.strip() for line in f if line.strip()}
    print(f"📖 Arquivo de registro encontrado. {len(processed_users)} usuários já foram processados.")
except FileNotFoundError:
    print("📖 Nenhum arquivo de registro encontrado. Começando do zero.")

# --- Etapa 2: Obter e Filtrar Lista de Usuários ---
try:
    print(f"\nBuscando lista de usuários na API...")
    response = requests.get(PRIMARY_API_URL)
    response.raise_for_status()
    all_items = response.json().get('broadcasts', {}).get('items', [])
    if not all_items: raise SystemExit("❌ A API não retornou nenhum usuário.")

    users_to_process = [item for item in all_items if item.get('username') not in processed_users]
    print(f"✅ API retornou {len(all_items)} usuários. Priorizando {len(users_to_process)} novos usuários.")
    if not users_to_process: raise SystemExit("🎉 Nenhum usuário novo para processar.")

except Exception as e:
    raise SystemExit(f"❌ Falha crítica na API: {e}")

# --- Etapa 3: Loop de Processamento ---
for i, item in enumerate(users_to_process):
    username = item.get('username')
    if not username: continue

    print(f"\n--- Processando novo usuário {i+1}/{len(users_to_process)}: {username} ---")

    # Define caminhos para arquivos temporários e finais
    temp_video_path = f"temp_{username}.mp4"
    temp_palette_path = f"temp_{username}_paleta.png"
    temp_sample_gif_path = f"temp_{username}_sample.gif"
    final_gif_path = os.path.join(DRIVE_FOLDER_PATH, f"{username}.gif")

    try:
        # Etapa 4: Obter URL .m3u8 (lógica anterior mantida)
        m3u8_url = None
        preview = item.get('preview', {})
        src_url = preview.get('src', '')
        if src_url and src_url.endswith('.m3u8'):
            m3u8_url = src_url
        else:
            secondary_url = SECONDARY_API_URL_TEMPLATE.format(username=username)
            user_response = requests.get(secondary_url)
            user_response.raise_for_status()
            webrtc_info = user_response.json().get('liveInfo', {}).get('webRTC', {})
            m3u8_url = webrtc_info.get('cdnURL') or webrtc_info.get('edgeURL')

        if not m3u8_url:
            print("❌ Nenhuma URL .m3u8 válida encontrada. Pulando.")
            continue
        print("🔗 URL .m3u8 encontrada.")

        # Etapa 5: Download do clipe de vídeo para análise
        print(f"🎬 Baixando clipe de {MAX_CLIP_SECONDS_TO_ANALYZE}s para análise...")
        os.system(f'ffmpeg -i "{m3u8_url}" -t {MAX_CLIP_SECONDS_TO_ANALYZE} -c copy -bsf:a aac_adtstoasc "{temp_video_path}" -y')
        if not os.path.exists(temp_video_path) or os.path.getsize(temp_video_path) == 0:
            raise ValueError("Falha ao baixar o vídeo de análise.")

        # --- Etapa 6: LÓGICA DE CONTROLE DE TAMANHO ---
        print("📏 Iniciando processo de calibração de tamanho...")

        # 6a. Gerar paleta a partir do vídeo inteiro
        os.system(f'ffmpeg -i "{temp_video_path}" -vf "palettegen" "{temp_palette_path}" -y')

        # 6b. Criar um GIF de amostra de 1 segundo
        os.system(f'ffmpeg -t 1 -i "{temp_video_path}" -i "{temp_palette_path}" -lavfi "fps={GIF_FPS},scale={GIF_WIDTH}:-1:flags=lanczos [x]; [x][1:v] paletteuse" "{temp_sample_gif_path}" -y')

        if not os.path.exists(temp_sample_gif_path) or os.path.getsize(temp_sample_gif_path) == 0:
            raise ValueError("Falha ao criar o GIF de amostra para calibração.")

        # 6c. Calcular a "taxa de dados" e a duração ideal
        sample_size_mb = os.path.getsize(temp_sample_gif_path) / (1024 * 1024)
        megabytes_per_second = sample_size_mb / 1.0  # Tamanho da amostra / duração da amostra (1s)

        if megabytes_per_second == 0:
            raise ValueError("Amostra com tamanho zero, impossível calcular a duração.")

        target_size_with_safety = TARGET_GIF_SIZE_MB * SAFETY_FACTOR
        estimated_duration = target_size_with_safety / megabytes_per_second
        final_duration = min(estimated_duration, MAX_CLIP_SECONDS_TO_ANALYZE) # Não exceder o clipe baixado

        print(f"📈 Amostra de 1s tem {sample_size_mb:.2f} MB. Taxa estimada: {megabytes_per_second:.2f} MB/s.")
        print(f"⏱️ Duração calculada para atingir {target_size_with_safety:.2f}MB: {final_duration:.2f} segundos.")

        if final_duration < MIN_GIF_DURATION:
            print(f"⚠️ Duração calculada ({final_duration:.2f}s) é menor que o mínimo ({MIN_GIF_DURATION}s). Pulando usuário para evitar GIF muito curto.")
            continue

        # 6d. Criar o GIF final com a duração calculada
        print(f"✨ Criando GIF final com {final_duration:.2f} segundos...")
        os.system(f'ffmpeg -t {final_duration} -i "{temp_video_path}" -i "{temp_palette_path}" -lavfi "fps={GIF_FPS},scale={GIF_WIDTH}:-1:flags=lanczos [x]; [x][1:v] paletteuse" "{final_gif_path}" -y')

        # Etapa 7: Registrar Sucesso
        final_gif_size_mb = os.path.getsize(final_gif_path) / (1024 * 1024)
        print(f"✅ Sucesso! GIF final criado com {final_gif_size_mb:.2f} MB.")
        with open(caminho_log, 'a') as f:
            f.write(f"{username}\n")
        print(f"✍️  '{username}' adicionado ao registro.")

    except Exception as e:
        print(f"❌ Ocorreu um erro ao processar '{username}': {e}.")
    finally:
        # Etapa 8: Limpar TODOS os arquivos temporários, independente de sucesso ou falha
        for f in [temp_video_path, temp_palette_path, temp_sample_gif_path]:
            if os.path.exists(f): os.remove(f)

print("\n\n🎉 Todos os NOVOS usuários foram processados. Automação concluída! 🎉")

Iniciando o processo de automação com controle de tamanho de arquivo...
📖 Nenhum arquivo de registro encontrado. Começando do zero.

Buscando lista de usuários na API...
✅ API retornou 867 usuários. Priorizando 867 novos usuários.

--- Processando novo usuário 1/867: classied3 ---
🔗 URL .m3u8 encontrada.
🎬 Baixando clipe de 30s para análise...
📏 Iniciando processo de calibração de tamanho...
📈 Amostra de 1s tem 1.72 MB. Taxa estimada: 1.72 MB/s.
⏱️ Duração calculada para atingir 14.25MB: 8.27 segundos.
✨ Criando GIF final com 8.27 segundos...
✅ Sucesso! GIF final criado com 14.29 MB.
✍️  'classied3' adicionado ao registro.

--- Processando novo usuário 2/867: Robnster99 ---
🔗 URL .m3u8 encontrada.
🎬 Baixando clipe de 30s para análise...
📏 Iniciando processo de calibração de tamanho...
📈 Amostra de 1s tem 2.07 MB. Taxa estimada: 2.07 MB/s.
⏱️ Duração calculada para atingir 14.25MB: 6.88 segundos.
✨ Criando GIF final com 6.88 segundos...
✅ Sucesso! GIF final criado com 13.68 MB.
✍️  'Rob